In [ ]:
pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=5cf2f5a9b56278946551295b38cc5af532e6af39f338b212fd7efb0bf265192c
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import anvil.server

anvil.server.connect("server_HHUBZ64I5PJ4XB6LCO3LXZIC-AGCG4YFEMUXOMWJS")

import numpy as np
import tensorflow as tf
import pandas as pd
from io import BytesIO
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import anvil.mpl_util
import csv

class ClassToken(tf.keras.layers.Layer):
  def __init__(self):
      super().__init__()

  def build(self, input_shape):
      w_init = tf.random_normal_initializer()
      self.w = tf.Variable(
          initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
          trainable = True
      )

  def call(self, inputs):
      batch_size = tf.shape(inputs)[0]
      hidden_dim = self.w.shape[-1]

      cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
      cls = tf.cast(cls, dtype=inputs.dtype)
      return cls


@anvil.server.callable
def get_input_file(file):
  file = file.get_bytes()
  input_df = pd.read_csv(BytesIO(file), header=None)
  input_df = np.array(input_df)
  max_val=0
  for i in range(input_df.shape[0]):
    if max_val<max(input_df[i]):
      max_val = max(input_df[i])
  if max_val>1:
      input_df = input_df/255
  return input_df

@anvil.server.callable
def check_csv(file):
    val = 0
    try:
        file_content = file.get_bytes().decode('utf-8')
        sample_data = '\n'.join(file_content.split('\n')[:5])

        csv.Sniffer().sniff(sample_data)
        val = 1
    except (csv.Error, UnicodeDecodeError):
        val=0

    return val

@anvil.server.callable
def get_input_file_shape(input_df):
  input_df = input_df.get_bytes()
  try:
    input_df = pd.read_csv(BytesIO(input_df), header=None)
  except:
    shape_val =0
  shape_val =0
  if input_df.shape == (28,28):
    shape_val =1
  return shape_val

@anvil.server.callable
def get_cnn_pred(input_df):
  input_df=np.array(input_df)
  df = input_df.reshape(1,28,28,1)

  cnn_model = load_model("CNN_Model_Final.h5")


  pred_cnn = np.argmax(cnn_model.predict(df), axis=1)
  return pred_cnn


@anvil.server.callable
def get_transformer_pred(input_df):
  input_df=np.array(input_df)
  df = input_df.reshape(1,28,28,1)

  with tf.keras.utils.custom_object_scope({'ClassToken': ClassToken}):
      transformer_model = load_model("Transformer_Model_Final.h5")

  pos_feed_test = np.array([list(range(16))]*1)

  ndata_test=1
  n=4
  m=4
  block_size=49
  x_test=df
  block_length=7

  x_test_ravel = np.zeros((ndata_test,n*m,block_size))
  for img in range(ndata_test):
      ind = 0
      for row in range(n):
          for col in range(m):
              x_test_ravel[img,ind,:] = x_test[img,(row*block_length):((row+1)*block_length),(col*block_length):((col+1)*block_length)].ravel()
              ind += 1


  pred_trans = np.argmax(transformer_model.predict([x_test_ravel,pos_feed_test]), axis=1)
  return pred_trans

@anvil.server.callable
def get_image_pred(file):
  file = file.get_bytes()
  input_df = pd.read_csv(BytesIO(file), header=None)
  max_val=0
  for i in range(input_df.shape[0]):
    if max_val<max(input_df[i]):
      max_val = max(input_df[i])
  if max_val>1:
      input_df = input_df/255
  plt.clf()
  plt.pcolor(1-input_df , cmap = 'gray' )
  plt.gca().invert_yaxis()
  plt.axis('off')
  return anvil.mpl_util.plot_image()

@anvil.server.callable
def get_prediction_image_link(pred_value):
  pred_value =  pred_value.replace('[','').replace(']','')
  return "_/theme/Prediction_"+pred_value+".png"

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
